In [8]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [9]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [22]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='gemma',
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0
    )
      
    return response

In [23]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [24]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [27]:
response = llm("What's the formula for energy?")

In [28]:
response.choices[0].message.content

'**E = mc^2**\n\nWhere:\n\n* **E** is the energy of the object\n* **m** is the mass of the object\n* **c** is the speed of light (299,792,458 meters per second)'

In [29]:
print(_)

**E = mc^2**

Where:

* **E** is the energy of the object
* **m** is the mass of the object
* **c** is the speed of light (299,792,458 meters per second)


In [31]:
response

ChatCompletion(id='chatcmpl-766', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='**E = mc^2**\n\nWhere:\n\n* **E** is the energy of the object\n* **m** is the mass of the object\n* **c** is the speed of light (299,792,458 meters per second)', role='assistant', function_call=None, tool_calls=None))], created=1720426586, model='gemma', object='chat.completion', service_tier=None, system_fingerprint='fp_ollama', usage=CompletionUsage(completion_tokens=59, prompt_tokens=0, total_tokens=59))

In [33]:
completion_tokens = response.usage.completion_tokens
print("Number of completion tokens:", completion_tokens)

Number of completion tokens: 59
